In [1]:
!pip install transformers accelerate evaluate datasets peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
import os
import torch
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModelForImageClassification, get_scheduler
from peft import AdaLoraConfig, get_peft_model

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model and dataset configuration
model_name_or_path = "google/vit-base-patch16-224-in21k"
dataset_name = "food101"
batch_size = 16
num_epochs = 8
learning_rate = 1e-4

# Load the dataset
dataset = load_dataset(dataset_name)
processor = AutoImageProcessor.from_pretrained(model_name_or_path)
normalize = transforms.Normalize(mean=processor.image_mean, std=processor.image_std)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize,
])

def preprocess_function(examples):
    examples["image"] = [transform(img.convert("RGB")) for img in examples["image"]]
    return examples

train_dataset = dataset["train"].with_transform(preprocess_function)
eval_dataset = dataset["validation"].with_transform(preprocess_function)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=4)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, num_workers=4)

total_training_steps = len(train_dataloader) * num_epochs

# Configure AdaLoRA
peft_config = AdaLoraConfig(
    init_r=12,
    target_r=8,
    beta1=0.85,
    beta2=0.85,
    tinit=200,
    tfinal=1000,
    deltaT=10,
    lora_alpha=32,
    lora_dropout=0.1,
    inference_mode=False,
    target_modules = ['query', 'key', 'value']
)

peft_config.total_step = total_training_steps

# Load the model and apply AdaLoRA
model = AutoModelForImageClassification.from_pretrained(model_name_or_path, num_labels=101)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.to(device)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * num_epochs
)

# Training loop with AdaLoRA update
global_step = 0
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        inputs, labels = batch["image"].to(device), batch["label"].to(device)
        optimizer.zero_grad()
        outputs = model(pixel_values=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        # Update AdaLoRA matrices and allocate budget
        model.base_model.update_and_allocate(global_step)
        global_step += 1
        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_dataloader)}")

    # Evaluate
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            inputs, labels = batch["image"].to(device), batch["label"].to(device)
            outputs = model(pixel_values=inputs)
            predictions = outputs.logits.argmax(dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    print(f"Validation Accuracy: {correct / total * 100:.2f}%")

# Save the fine-tuned model
model.save_pretrained(f"{model_name_or_path}_adalora_fine_tuned")

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 663,984 || all params: 86,540,345 || trainable%: 0.7673


Training Epoch 1:  61%|██████    | 2880/4735 [04:39<02:58, 10.40it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Training Epoch 1: 100%|██████████| 4735/4735 [07:39<00:00, 10.31it/s]


Epoch 1: Loss = 4.2067440153804725


Evaluating:  14%|█▍        | 228/1579 [00:11<01:04, 20.94it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Evaluating:  34%|███▍      | 543/1579 [00:26<00:49, 21.00it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Evaluating: 100%|██████████| 1579/1579 [01:15<00:00, 20.85it/s]


...

Validation Accuracy: 73.10%


Training Epoch 8:  43%|████▎     | 2018/4735 [03:15<04:23, 10.32it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Training Epoch 8: 100%|██████████| 4735/4735 [07:39<00:00, 10.30it/s]


Epoch 8: Loss = 3.4043755804221005


Evaluating:  14%|█▍        | 228/1579 [00:11<01:04, 21.05it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Evaluating:  34%|███▍      | 543/1579 [00:26<00:49, 20.73it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Evaluating: 100%|██████████| 1579/1579 [01:15<00:00, 20.84it/s]


Validation Accuracy: 73.60%
